In [19]:
import os
import pandas as pd


directory_path = "/Users/victoriamishchenko/Documents/Входящие"
df_list = []

# Фильтрация файлов - работаем только с csv-расширением
for filename in os.listdir(directory_path):
    name, extension = os.path.splitext(filename)
    
    if extension == '.csv':
        file_path = os.path.join(directory_path, filename)
        df_temp = pd.read_csv(file_path, encoding='1251', sep=';')
        df_list.append(df_temp)


if df_list:
    df = pd.concat(df_list) # concat для соеднинения датафреймов одинаковой структуры
else:
    print("csv-файлы не найдены")

In [27]:
# Переименование столбцов с заменой пробелов на _
renamed_cols = list(map(
    lambda x: x.replace(" ", "_"), 
    [
        "Дата",
        "Номер",
        "Сумма",
        "Статус",
        "Примечание",
        "Комментарий",
        "Контрагент",
        "ИНН/КПП",
        "Организация",
        "ИНН/КПП",
        "Тип документа",
        "Имя файла",
        "Дата",
        "Номер 1",
        "Сумма 1",
        "Сумма НДС",
        "Ответственный",
        "Подразделение",
        "Код",
        "Дата",
        "Время",
        "Тип пакета",
        "Идентификатор пакета",
        "Запущено в обработку",
        "Получено контрагентом",
        "Завершено",
        "Увеличение суммы",
        "НДC",
        "Уменьшение суммы",
        "НДС"
    ]))

df.columns = renamed_cols

In [212]:
# Выгрузка данных из аптек
directory_path = "/Users/victoriamishchenko/Documents/Аптеки/csv/correct"
df_dict = {}

# Фильтрация файлов - работаем только с csv-расширением
for filename in os.listdir(directory_path):
    name, extension = os.path.splitext(filename)
    
    if extension == '.csv':
        file_path = os.path.join(directory_path, filename)
        df_dict[name] = pd.read_csv(file_path, encoding='1251', sep=';')
        
        print(f"Создан датафрейм {name}.")

Создан датафрейм А123.
Создан датафрейм 366.


In [ ]:
# Загрузка данных из аптек в отдельные датафреймы
df_apt123 = df_dict['А123']
df_apt366 = df_dict['366']

In [216]:
import numpy as np

# Создание новых столбцов
new_columns = ["Номер счет-фактуры",
               "Сумма счет-фактуры",
               "Дата счет-фактуры",
               "Сравнение дат",
]

# Список типов учитываемых документов
docs_types = ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]

def data_processing(df_name, df_sbis):
    """
    Обрабатывает df_name, сопоставляя записи с df_sbis 
    по номеру накладной: добавляет/заполняет столбцы с номером, 
    суммой и датой счет‑фактуры, выполняет сравнение дат 
    и возвращает таблицу с нужными колонками.

    Параметры:
    df_name: pandas.DataFrame с данными из аптеки для модификации
    df_sbis: pandas.DataFrame с данными из СБИСа для поиска документов
    """

    # Создание новых столбцов
    for col in new_columns:
        df_name[col] = ''

    
    # Добавляем '/15' к номеру накладной для ЕАПТЕКИ
    df_name['Номер накладной'] = (
                np.where(df_name['Поставщик'].str.contains('ЕАПТЕКА', na=False), 
                         df_name['Номер накладной'] + '/15',  
                         df_name['Номер накладной'])
    )

    # Работа со строками через iterrows
    for i, row in df_name.iterrows():
        num_nakl = row['Номер накладной']
        
        # 2. Поиск всех записей из СБИСа по номеру накладной
        found = df_sbis[df_sbis['Номер'] == num_nakl]
        
        # 3. Фильтрация по конкретным типам документов
        found = found[found['Тип_документа'].isin(docs_types)]
        
        # 4. Если ничего не найдено - идем дальше
        if found.empty:
            continue
        
        # Выбор первого документа, если нашлось несколько
        first = found.iloc[0]
        
        # 5. Сохраняем значения
        number_value = first['Номер']
        summ_value = first['Сумма']
        
        # В датафрейме из СБИСа 3 столбца "Дата" - отбираем первый
        date_columns = [col for col in df_sbis.columns if col == 'Дата']
        if date_columns:
            date_value = first[date_columns[0]]
            # Преобразуем в скаляр если это Series
            if hasattr(date_value, 'iloc'):
                date_value = date_value.iloc[0]
        else:
            date_value = None
        
        
        # 5, 7. Если найдено, то сохраняем нужные значения
        df_name.at[i, 'Номер счет-фактуры'] = number_value
        df_name.at[i, 'Сумма счет-фактуры'] = summ_value
                
        # 6. Устанавливаем конкретный формат даты
        
        date_fact = pd.to_datetime(date_value, dayfirst=True, errors='coerce')  # coerce вернет NaT в случае некорректной даты
        
        df_name.at[i, 'Дата счет-фактуры'] = (
            date_fact.strftime('%d.%m.%Y') if pd.notnull(date_fact) else None
        )
    
        # 8. Сравнение дат
        date_nakl = pd.to_datetime(row['Дата накладной'], dayfirst=True,  errors='coerce')  # Преобразование даты накладной (из аптеки)
       
        if pd.notnull(date_fact) and pd.notnull(date_nakl):
            if date_nakl.date() != date_fact.date():
                df_name.at[i, 'Сравнение дат'] = 'Не совпадает!'
            else:
                df_name.at[i, 'Сравнение дат'] = ''
        else:
            df_name.at[i, 'Сравнение дат'] = 'Ошибка в дате'
    
    # Переопределяем столбцы
    final_columns = ['№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',
    'Дата приходного документа', 'Номер приходного документа',
    'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',
    'Сумма счет-фактуры', 'Кол-во',
    'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
    'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат']

    # Оставляем только существующие колонки, если есть другие
    existing_columns = [col for col in final_columns if col in df_name.columns]
    
    return df_name[existing_columns]

In [217]:
from datetime import datetime

def save_results(df_result, original_filename, base_output_dir="Результат"):
    """
    Сохраняет DataFrame в формате .xlsx в папку с текущей датой.
    
    Parameters:
    df_result: обработанный pandas.DataFrame для сохранения
    original_filename: str с исходным именем файла (без расширения)
    base_output_dir: str с базовой папкой для результатов
    """
    
    # Получаем сегодняшнюю дату в формате ДД-ММ-ГГГ
    today_date = datetime.now().strftime("%d-%m-%Y")
    
    # Формируем полный путь: Результат/{дата}/
    output_dir = os.path.join(base_output_dir, today_date) 
    
    # Создаем папки, если они не существуют
    os.makedirs(output_dir, exist_ok=True)
    
    # Формируем имя файла
    output_filename = f"{original_filename} - результат.xlsx"
    output_path = os.path.join(output_dir, output_filename)
    
    # Сохраняем в Excel
    df_result.to_excel(output_path, index=False)
    
    print(f"Файл сохранен: {output_path}")

In [218]:
# Обработка данных из аптеки
df_123_upd = data_processing(df_apt123, df)
df_366_upd = data_processing(df_apt366, df)

In [226]:
# Сохранение обработанного датафрейма
save_results(df_123_upd, "А123")  
save_results(df_366_upd, "366")

Файл сохранен: Результат/06-11-2025/А123 - результат.xlsx
Файл сохранен: Результат/06-11-2025/366 - результат.xlsx


In [227]:
# Пример: чтение файла
df366 = pd.read_excel("/Users/victoriamishchenko/Результат/06-11-2025/366 - результат.xlsx")
df366 = df366.fillna('') # Замена NaN на пустую строку

In [236]:
df366.head()

,№ п/п,Штрих-код партии,Наименование товара,Поставщик,Дата приходного документа,Номер приходного документа,Дата накладной,Номер накладной,Номер счет-фактуры,Сумма счет-фактуры,Кол-во,Сумма в закупочных ценах без НДС,Ставка НДС поставщика,Сумма НДС,Сумма в закупочных ценах с НДС,Дата счет-фактуры,Сравнение дат
0,1,200200060311,СФМ ПЕРЧАТКИ СМОТР. Н/СТЕР. ЛАТ. Р.S №100 (50П...,Катрен г.Химки,26.05.2021,2002088.0,25.05.2021,4665120-30,4665120-30,"16 728,87",0.040000,33.65,10%,3.37,37.02,25.05.2021,
1,2,200801594420,КОСМОПОР ПОВЯЗКА 10СМХ6СМ №25 П/ОПЕР. САМОКЛ. ...,Здравсервис,25.03.2020,9000479.8,25.03.2020,E7914339/27806739,E7914339/27806739,"3 395,94",0.040000,27.58,10%,2.76,30.34,25.03.2020,
2,3,200200075091,ХАРТМАНН БРАНОЛИНД H ПОВЯЗКА СТЕР. 10Х20СМ. №3...,Протек,01.09.2021,2003451.0,01.09.2021,164621235-001,164621235-001,"13 129,99",0.066667,145.28,10%,14.53,159.81,01.09.2021,
3,4,200801670938,КОСМОПОР Е ПОВЯЗКА 20СМХ8СМ №25 П/ОПЕР. САМОКЛ...,Протек,17.06.2020,9000628.8,17.06.2020,132215190-001,132215190-001,"11 893,01",0.080000,47.07,10%,4.71,51.78,17.06.2020,
4,5,200801489808,"ФОКУСИН 0,4МГ. №90 КАПС. МОДИФ.ВЫСВ. /ЗЕНТИВА/...",Протек,12.12.2019,9001590.8,12.12.2019,120939343-001,120939343-001,"3 219,81",0.111111,96.37,10%,9.64,106.00,12.12.2019,
